In [6]:
import pandas as pd
import numpy as np

# Load the metadata, weather, and electricity consumption data
metadata = pd.read_csv('metadata.csv')
weather = pd.read_csv('weather.csv')
electricity = pd.read_csv('electricity_cleaned.csv')

# Handle missing values in the weather data
weather['precipDepth1HR'] = weather['precipDepth1HR'].replace(-1.0, np.nan)
weather['precipDepth6HR'] = weather['precipDepth6HR'].replace(-1.0, np.nan)

# Interpolate missing values for cloudCoverage
weather['cloudCoverage'] = weather['cloudCoverage'].interpolate()

# Replace windDirection and windSpeed where both are 0.0 with NaN
weather.loc[(weather['windDirection'] == 0.0) & (weather['windSpeed'] == 0.0), ['windDirection', 'windSpeed']] = np.nan

# Convert timestamps to datetime objects and set as index where appropriate
weather['timestamp'] = pd.to_datetime(weather['timestamp'], errors='coerce')
electricity['timestamp'] = pd.to_datetime(electricity['timestamp'], errors='coerce')

# Set timestamp as index for electricity data
electricity.set_index('timestamp', inplace=True)

# Display first few rows to verify the preprocessing
print("Metadata:\n", metadata.head())
print("\nWeather Data:\n", weather.head())
print("\nElectricity Data:\n", electricity.head())

Metadata:
                  building_id  site_id  building_id_kaggle  site_id_kaggle  \
0       Panther_lodging_Dean  Panther                 NaN             0.0   
1     Panther_lodging_Shelia  Panther                 NaN             0.0   
2      Panther_lodging_Ricky  Panther                 NaN             0.0   
3  Panther_education_Rosalie  Panther                 0.0             0.0   
4    Panther_education_Misty  Panther                 1.0             0.0   

     primaryspaceusage sub_primaryspaceusage    sqm     sqft        lat  \
0  Lodging/residential        Residence Hall  508.8   5477.0  28.517689   
1  Lodging/residential        Residence Hall  929.0  10000.0  28.517689   
2  Lodging/residential        Residence Hall  483.1   5200.0  28.517689   
3            Education              Research  690.5   7432.0  28.517689   
4            Education              Research  252.7   2720.0  28.517689   

         lng  ... yearbuilt date_opened numberoffloors occupants  \
0 -81.3

In [8]:
# 1. How many unique sites and buildings are in the dataset?
unique_sites = metadata['site_id'].nunique()
unique_buildings = metadata['building_id'].nunique()

print(f"1. How many unique sites are in the dataset? {unique_sites}")
print(f"How many unique buildings are in the dataset? {unique_buildings}")

# 2. What is the date range of the weather data?
weather['timestamp'] = pd.to_datetime(weather['timestamp'], errors='coerce')
date_min = weather['timestamp'].min()
date_max = weather['timestamp'].max()

print(f"\n2. What is the date range of the weather data? From {date_min} to {date_max}")

# 3. How many weather variables are recorded, and what are they?
weather_columns = weather.columns.drop(['timestamp', 'site_id'])
num_weather_variables = len(weather_columns)
weather_variables = list(weather_columns)

print(f"\n3. How many weather variables are recorded? {num_weather_variables}")
print(f"What are the weather variables? {weather_variables}")

# 4. What percentage of values are missing in the cloudCoverage column of the weather data?
missing_cloud_coverage = weather['cloudCoverage'].isna().sum()
total_cloud_coverage = len(weather['cloudCoverage'])
percent_missing_cloud_coverage = (missing_cloud_coverage / total_cloud_coverage) * 100

print(f"\n4. What percentage of values are missing in the cloudCoverage column? {percent_missing_cloud_coverage:.2f}%")

# Checking for missing values in metadata and weather data
metadata_missing_values = metadata.isna().sum()
weatherdata_missing_values = weather.isna().sum()

# Note: Results are giving me way too many missing values, what is a systemic way of approaching this?

print(metadata_missing_values, weatherdata_missing_values)


1. How many unique sites are in the dataset? 19
How many unique buildings are in the dataset? 1636

2. What is the date range of the weather data? From 2016-01-01 00:00:00 to 2017-12-31 23:00:00

3. How many weather variables are recorded? 8
What are the weather variables? ['airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']

4. What percentage of values are missing in the cloudCoverage column? 0.00%
building_id                 0
site_id                     0
building_id_kaggle        187
site_id_kaggle             37
primaryspaceusage          21
sub_primaryspaceusage      21
sqm                         0
sqft                        0
lat                       237
lng                       237
timezone                    0
electricity                58
hotwater                 1451
chilledwater             1081
steam                    1266
water                    1490
irrigation               1599
s